In [1]:
import tensorflow as tf 
import numpy as np 
from numpy import random
from keras.models import Sequential 
from keras.utils import np_utils 
from keras.layers import Dense,Activation,LSTM,Dropout,AveragePooling3D 
import pandas as pd 
import matplotlib.pyplot as plt 
from tensorflow import keras
import os
import glob


In [2]:
df=pd.read_csv('data3.csv')
df.head()
print(df)

     Unnamed: 0    0    1    2    3    4    5    6   7    8  ...  4087  4088  \
0             0  -33  -55  -78  -81  -69  -48  -31 -13   -4  ...   -47   -45   
1             1   88   77   69   56   45   35   21  -8  -33  ...   -69   -32   
2             2    8    9   10   11    8    3    0  -1   -4  ...   -29   -30   
3             3   -7  -12  -14  -21  -31  -42  -48 -47  -53  ...   -10   -10   
4             4 -142 -142 -133 -126 -123 -112 -100 -91  -93  ...  -148  -150   
..          ...  ...  ...  ...  ...  ...  ...  ...  ..  ...  ...   ...   ...   
495         495  -13   -4    2   21   25   26   11   6    4  ...   -37   -14   
496         496 -133 -144 -113  -74  -41  -41  -61 -89 -107  ...   -88   -81   
497         497  -20  -12   -6    6    8   -2  -22 -39  -39  ...     1     7   
498         498   14   25   31   36   33   33   27  27   21  ...    -6    -7   
499         499   18   34   45   37   35   34   38  38   43  ...  -110   -88   

     4089  4090  4091  4092  4093  4094

In [3]:
df = pd.read_csv('data3.csv')
df['y']=1 ## mmake all 1's then make normal subjects data to be 0's
df['y'][200:300]=0 ## O set--normal subjects
df['y'][400:500]=0  ## Z set--normal subjects
df.head()
print(df)

x=df.values
x_signals=x[:,1:(x.shape[1]-2)]
x_signals = np.asarray(x_signals).astype(np.float32)

     Unnamed: 0    0    1    2    3    4    5    6   7    8  ...  4088  4089  \
0             0  -33  -55  -78  -81  -69  -48  -31 -13   -4  ...   -45   -42   
1             1   88   77   69   56   45   35   21  -8  -33  ...   -32    24   
2             2    8    9   10   11    8    3    0  -1   -4  ...   -30   -44   
3             3   -7  -12  -14  -21  -31  -42  -48 -47  -53  ...   -10    -9   
4             4 -142 -142 -133 -126 -123 -112 -100 -91  -93  ...  -150  -161   
..          ...  ...  ...  ...  ...  ...  ...  ...  ..  ...  ...   ...   ...   
495         495  -13   -4    2   21   25   26   11   6    4  ...   -14     4   
496         496 -133 -144 -113  -74  -41  -41  -61 -89 -107  ...   -81   -70   
497         497  -20  -12   -6    6    8   -2  -22 -39  -39  ...     7    20   
498         498   14   25   31   36   33   33   27  27   21  ...    -7    -2   
499         499   18   34   45   37   35   34   38  38   43  ...   -88   -81   

     4090  4091  4092  4093  4094  4095

In [4]:
n_sample_points=400 #for every 2.3s segment the data
n_seg=x_signals.shape[1]/n_sample_points
data_seg_list=[]
data_seg_label_list=[]
for sig in range(x_signals.shape[0]):
  individual_signal=x_signals[sig,:]
  label=df['y'][sig]
  for seg in range(0,int(n_seg)):
    data_segmented=individual_signal[(seg*n_sample_points):((seg+1)*n_sample_points)]
    data_seg_list.append(data_segmented)
    data_seg_label_list.append(label)

In [5]:
data_seg=np.asarray(data_seg_list)
data_seg_label_=np.asarray(data_seg_label_list)
data_seg_label=np.empty((data_seg_label_.shape[0],1))
data_seg_label[:,0]=data_seg_label_


indices=np.arange(0,data_seg.shape[0])

shuffle_indices=random.permutation(indices)

In [6]:
###spilitting train and test by ratio 80:20
train_ratio=0.8
split_indices=data_seg.shape[0]*0.8

In [7]:
train_data=data_seg[shuffle_indices[0:int(split_indices)],:]
train_labels=data_seg_label[shuffle_indices[0:int(split_indices)],:]
test_data=data_seg[shuffle_indices[(int(split_indices)):-1],:]
test_labels=data_seg_label[shuffle_indices[(int(split_indices)):-1],:]

In [8]:
## adaptation of https://www.sciencedirect.com/science/article/pii/S0957417418302513#bib0050
#Normalise the data using z-score normalisation
train_data_normalised=(train_data-train_data.mean())/train_data.std()
train_data_normalised = train_data_normalised.reshape((-1, 1, 400))
# train_data_normalised = train_data_normalised.reshape((1, 400, 4000))
# train_data_normalised = train_data_normalised.reshape((-1, 400, 1))
#conv-1
inputs = tf.keras.Input(shape=(None,400))
conv1 = tf.keras.layers.Conv1D(24, (5),strides=1, padding='same')(inputs)
bn1=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv1)
rel1 = tf.keras.layers.Activation('relu')(bn1)

#conv-2
conv2 = tf.keras.layers.Conv1D(16, (3),strides=1, padding='same')(rel1)
bn2=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv2)
rel2 = tf.keras.layers.Activation('relu')(bn2)

#conv-3
conv3 = tf.keras.layers.Conv1D(8, (3),strides=1, padding='same')(rel2)
bn3=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv3)
rel3 = tf.keras.layers.Activation('relu')(bn3)

#FC-1
fc1=tf.keras.layers.Dense(20, activation='relu', use_bias=True)(rel3)

#dropout
do=tf.keras.layers.Dropout(rate=0.1)(fc1)

#FC-2
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(do)
##build the model
models = tf.keras.Model(inputs=inputs, outputs=outputs)

In [11]:

opt=tf.keras.optimizers.Adam(learning_rate=0.001)
models.compile(optimizer=opt, loss="binary_crossentropy", metrics=[ "acc"])
models.fit(train_data_normalised,train_labels,1,epochs=50)

Epoch 1/50
4000/4000 [==============================] - 4s 793us/step - loss: 0.6931 - acc: 0.4020
Epoch 2/50
4000/4000 [==============================] - 3s 791us/step - loss: 0.6931 - acc: 0.4007
Epoch 3/50
4000/4000 [==============================] - 3s 780us/step - loss: 0.6931 - acc: 0.3900
Epoch 4/50
4000/4000 [==============================] - 3s 795us/step - loss: 0.6931 - acc: 0.4076
Epoch 5/50
4000/4000 [==============================] - 3s 795us/step - loss: 0.6931 - acc: 0.4118
Epoch 6/50
4000/4000 [==============================] - 3s 791us/step - loss: 0.6931 - acc: 0.3971
Epoch 7/50
4000/4000 [==============================] - 3s 787us/step - loss: 0.6931 - acc: 0.3978
Epoch 8/50
4000/4000 [==============================] - 3s 782us/step - loss: 0.6931 - acc: 0.3903
Epoch 9/50
4000/4000 [==============================] - 3s 810us/step - loss: 0.6931 - acc: 0.4024
Epoch 10/50
4000/4000 [==============================] - 3s 794us/step - loss: 0.6931 - acc: 0.4028
Epoch 11/

In [12]:
test_data_1 = test_data.reshape((-1, 1, 400))
models.evaluate(test_data_1, test_labels,1)

999/999 [==============================] - 1s 473us/step - loss: 7.6786 - acc: 0.5055


[7.678557872772217, 0.5055055022239685]

In [14]:
classify=models.predict(test_data_1)


In [15]:

opt=tf.keras.optimizers.SGD(learning_rate=0.01)
models.compile(optimizer=opt, loss="mse", metrics=[ "acc"])
models.fit(train_data_normalised,train_labels,1,epochs=50)

Epoch 1/50
4000/4000 [==============================] - 4s 776us/step - loss: 0.2500 - acc: 0.4018
Epoch 2/50
4000/4000 [==============================] - 4s 1ms/step - loss: 0.2500 - acc: 0.4058
Epoch 3/50
4000/4000 [==============================] - 3s 664us/step - loss: 0.2500 - acc: 0.3985
Epoch 4/50
4000/4000 [==============================] - 3s 741us/step - loss: 0.2500 - acc: 0.4003
Epoch 5/50
4000/4000 [==============================] - 3s 768us/step - loss: 0.2500 - acc: 0.4016
Epoch 6/50
4000/4000 [==============================] - 3s 676us/step - loss: 0.2500 - acc: 0.4018
Epoch 7/50
4000/4000 [==============================] - 3s 690us/step - loss: 0.2500 - acc: 0.3940
Epoch 8/50
4000/4000 [==============================] - 3s 701us/step - loss: 0.2500 - acc: 0.3953
Epoch 9/50
4000/4000 [==============================] - 3s 713us/step - loss: 0.2500 - acc: 0.3926
Epoch 10/50
4000/4000 [==============================] - 3s 700us/step - loss: 0.2500 - acc: 0.4111
Epoch 11/50

In [16]:
test_data_2 = test_data.reshape((-1, 1, 400))
models.evaluate(test_data_2, test_labels,1)

999/999 [==============================] - 1s 506us/step - loss: 0.5000 - acc: 0.5045


[0.5, 0.5045045018196106]

In [17]:

opt=tf.keras.optimizers.Adam(learning_rate=0.01)
models.compile(optimizer=opt, loss="mse", metrics=[ "acc"])
models.fit(train_data_normalised,train_labels,1,epochs=50)

Epoch 1/50
4000/4000 [==============================] - 4s 761us/step - loss: 0.2500 - acc: 0.3964
Epoch 2/50
4000/4000 [==============================] - 3s 784us/step - loss: 0.2500 - acc: 0.4226
Epoch 3/50
4000/4000 [==============================] - 3s 773us/step - loss: 0.2500 - acc: 0.4001
Epoch 4/50
4000/4000 [==============================] - 3s 757us/step - loss: 0.2500 - acc: 0.4089
Epoch 5/50
4000/4000 [==============================] - 3s 758us/step - loss: 0.2500 - acc: 0.4041
Epoch 6/50
4000/4000 [==============================] - 3s 760us/step - loss: 0.2500 - acc: 0.4009
Epoch 7/50
4000/4000 [==============================] - 3s 762us/step - loss: 0.2500 - acc: 0.4167
Epoch 8/50
4000/4000 [==============================] - 3s 780us/step - loss: 0.2500 - acc: 0.4141
Epoch 9/50
4000/4000 [==============================] - 3s 813us/step - loss: 0.2500 - acc: 0.4020
Epoch 10/50
4000/4000 [==============================] - 3s 780us/step - loss: 0.2500 - acc: 0.4047
Epoch 11/

In [18]:
test_data_3 = test_data.reshape((-1, 1, 400))
models.evaluate(test_data_3, test_labels,1)

999/999 [==============================] - 1s 446us/step - loss: 0.5000 - acc: 0.5055


[0.5, 0.5055055022239685]

In [20]:
train_data_normalised=(train_data-train_data.mean())/train_data.std()
train_data_normalised = train_data_normalised.reshape((-1, 1, 400))
# train_data_normalised = train_data_normalised.reshape((1, 400, 4000))
# train_data_normalised = train_data_normalised.reshape((-1, 400, 1))
#conv-1
inputs = tf.keras.Input(shape=(None,400))
conv1 = tf.keras.layers.Conv1D(24, (5),strides=1, padding='same')(inputs)
bn1=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv1)
rel1 = tf.keras.layers.Activation('sigmoid')(bn1)

#conv-2
conv2 = tf.keras.layers.Conv1D(16, (3),strides=1, padding='same')(rel1)
bn2=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv2)
rel2 = tf.keras.layers.Activation('sigmoid')(bn2)

#conv-3
conv3 = tf.keras.layers.Conv1D(8, (3),strides=1, padding='same')(rel2)
bn3=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv3)
rel3 = tf.keras.layers.Activation('sigmoid')(bn3)

#FC-1
fc1=tf.keras.layers.Dense(20, activation='sigmoid', use_bias=True)(rel3)

#dropout
do=tf.keras.layers.Dropout(rate=0.1)(fc1)

#FC-2
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(do)
##build the model
models2 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [21]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
models.compile(optimizer=opt, loss="binary_crossentropy", metrics=[ "acc"])
models.fit(train_data_normalised,train_labels,1,epochs=50)

Epoch 1/50
4000/4000 [==============================] - 4s 773us/step - loss: 0.6931 - acc: 0.3874
Epoch 2/50
4000/4000 [==============================] - 3s 764us/step - loss: 0.6931 - acc: 0.3927
Epoch 3/50
4000/4000 [==============================] - 3s 792us/step - loss: 0.6931 - acc: 0.3944
Epoch 4/50
4000/4000 [==============================] - 3s 781us/step - loss: 0.6931 - acc: 0.4080
Epoch 5/50
4000/4000 [==============================] - 3s 780us/step - loss: 0.6931 - acc: 0.4122
Epoch 6/50
4000/4000 [==============================] - 3s 786us/step - loss: 0.6931 - acc: 0.4056
Epoch 7/50
4000/4000 [==============================] - 3s 777us/step - loss: 0.6931 - acc: 0.4112
Epoch 8/50
4000/4000 [==============================] - 3s 775us/step - loss: 0.6931 - acc: 0.4099
Epoch 9/50
4000/4000 [==============================] - 3s 777us/step - loss: 0.6931 - acc: 0.4043
Epoch 10/50
4000/4000 [==============================] - 3s 771us/step - loss: 0.6931 - acc: 0.3900
Epoch 11/

In [22]:
test_data_4 = test_data.reshape((-1, 1, 400))
models.evaluate(test_data_4, test_labels,1)

999/999 [==============================] - 1s 451us/step - loss: 7.6786 - acc: 0.5055


[7.678557872772217, 0.5055055022239685]

In [23]:
train_data_normalised=(train_data-train_data.mean())/train_data.std()
train_data_normalised = train_data_normalised.reshape((-1, 1, 400))
# train_data_normalised = train_data_normalised.reshape((1, 400, 4000))
# train_data_normalised = train_data_normalised.reshape((-1, 400, 1))
#conv-1
inputs = tf.keras.Input(shape=(None,400))
conv1 = tf.keras.layers.Conv1D(24, (5),strides=1, padding='same')(inputs)
bn1=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv1)
rel1 = tf.keras.layers.Activation('relu')(bn1)

#conv-2
conv2 = tf.keras.layers.Conv1D(16, (3),strides=1, padding='same')(rel1)
bn2=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv2)
rel2 = tf.keras.layers.Activation('relu')(bn2)

#conv-3
conv3 = tf.keras.layers.Conv1D(8, (3),strides=1, padding='same')(rel2)
bn3=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv3)
rel3 = tf.keras.layers.Activation('relu')(bn3)

#FC-1
fc1=tf.keras.layers.Dense(20, activation='sigmoid', use_bias=True)(rel3)

#dropout
do=tf.keras.layers.Dropout(rate=0.1)(fc1)

#FC-2
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(do)
##build the model
models3 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [25]:
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
models.compile(optimizer=opt, loss="logcosh", metrics=[ "acc"])
models.fit(train_data_normalised,train_labels,1,epochs=50)

Epoch 1/50
4000/4000 [==============================] - 3s 753us/step - loss: 0.1201 - acc: 0.4030
Epoch 2/50
4000/4000 [==============================] - 3s 755us/step - loss: 0.1201 - acc: 0.3946
Epoch 3/50
4000/4000 [==============================] - 3s 748us/step - loss: 0.1201 - acc: 0.4023
Epoch 4/50
4000/4000 [==============================] - 3s 761us/step - loss: 0.1201 - acc: 0.3984
Epoch 5/50
4000/4000 [==============================] - 3s 756us/step - loss: 0.1201 - acc: 0.4062
Epoch 6/50
4000/4000 [==============================] - 3s 769us/step - loss: 0.1201 - acc: 0.3900
Epoch 7/50
4000/4000 [==============================] - 3s 767us/step - loss: 0.1201 - acc: 0.3960
Epoch 8/50
4000/4000 [==============================] - 3s 761us/step - loss: 0.1201 - acc: 0.4039
Epoch 9/50
4000/4000 [==============================] - 3s 755us/step - loss: 0.1201 - acc: 0.3959
Epoch 10/50
4000/4000 [==============================] - 3s 753us/step - loss: 0.1201 - acc: 0.4110
Epoch 11/

In [26]:
test_data_5 = test_data.reshape((-1, 1, 400))
models.evaluate(test_data_5, test_labels,1)


999/999 [==============================] - 1s 472us/step - loss: 0.2169 - acc: 0.5055


[0.21688927710056305, 0.5055055022239685]

In [29]:
opt=tf.keras.optimizers.Nadam(learning_rate=0.001)
models.compile(optimizer=opt, loss="logcosh", metrics=[ "acc"])
models.fit(train_data_normalised,train_labels,1,epochs=50)

Epoch 1/50
4000/4000 [==============================] - 5s 953us/step - loss: 0.1201 - acc: 0.4019
Epoch 2/50
4000/4000 [==============================] - 4s 952us/step - loss: 0.1201 - acc: 0.3967
Epoch 3/50
4000/4000 [==============================] - 4s 956us/step - loss: 0.1201 - acc: 0.3860
Epoch 4/50
4000/4000 [==============================] - 4s 954us/step - loss: 0.1201 - acc: 0.4008
Epoch 5/50
4000/4000 [==============================] - 4s 984us/step - loss: 0.1201 - acc: 0.4030
Epoch 6/50
4000/4000 [==============================] - 4s 987us/step - loss: 0.1201 - acc: 0.4088
Epoch 7/50
4000/4000 [==============================] - 4s 1ms/step - loss: 0.1201 - acc: 0.4089
Epoch 8/50
4000/4000 [==============================] - 4s 1ms/step - loss: 0.1201 - acc: 0.3971
Epoch 9/50
4000/4000 [==============================] - 4s 956us/step - loss: 0.1201 - acc: 0.4067
Epoch 10/50
4000/4000 [==============================] - 4s 968us/step - loss: 0.1201 - acc: 0.4237
Epoch 11/50
4

In [30]:
test_data_6 = test_data.reshape((-1, 1, 400))
models.evaluate(test_data_6, test_labels,1)


999/999 [==============================] - 1s 481us/step - loss: 0.2169 - acc: 0.5045


[0.21688927710056305, 0.5045045018196106]